In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob, os

import sys
sys.path.append('../..')

from sdt_dask.dask_tool.sdt_dask import SDTDask

In [3]:
SDTDask?

Init signature: SDTDask(data_plug, client, **keywords)
Docstring:      <no docstring>
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dask_tool/sdt_dask.py
Type:           type
Subclasses:     

## Local Client example 
 use PVDAQ, local csv dataplug and local client

### Instantiate a pvdaq data plug


In [4]:
from sdt_dask.dataplugs.pvdaq_plug import PVDAQPlug

In [5]:
PVDAQPlug?

Init signature: PVDAQPlug(api_key='DEMO_KEY', power_col='ac_power')
Docstring:     
Dataplug class for retrieving data from the PVDAQ DB.
Note that the DEMO_KEY has a rate limit of 30/h, 50/d per IP address.
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dataplugs/pvdaq_plug.py
Type:           type
Subclasses:     

In [6]:
pvdaq_data_plug = PVDAQPlug()

### Set a local User-defined Client

In [7]:
from dask.distributed import Client

In [8]:
n_workers = 1
threads_per_worker = 1
total_system_memory = 8
memory_per_worker = total_system_memory / n_workers
local_client = Client(processes=False, memory_spill_fraction=False, memory_pause_fraction=False, memory_target_fraction=0.8, n_workers=n_workers, threads_per_worker=threads_per_worker, memory_limit=f"{memory_per_worker}GiB")
local_client.dashboard_link

2024-03-21 15:45:30,578 - distributed.worker.memory - WARNING - Ignoring provided memory limit 8.0GiB due to system memory limit of 7.66 GiB
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_target_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.target instead
  warnings.warn(
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_spill_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.spill instead
  warnings.warn(
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_pause_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.pause 

'http://172.21.212.11:8787/status'

(34, 2011)
Pulling site ID 34 for year 2011...
[============================================================] 100.0% ...queries complete in 1.7 seconds       

(51, 2012)
Pulling site ID 51 for year 2012...
[============================================================] 100.0% ...queries complete in 2.2 seconds       

total time: 15.24 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              3.40s
Cleaning                   2.46s
Filtering/Summarizing      9.37s
    Data quality           0.11s
    Clear day detect       0.52s
    Clipping detect        7.89s
    Capacity change detect 0.85s

total time: 14.11 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              5.73s
Cleaning                   2.24s
Filtering/Summarizing      6.15s
    Data quality           0.21s
    Clear day detect       0.70s
    Clipping detect        4.24s
    Capacity change detect 1.00s

(35, 2015)
P

/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


[============================================================] 100.0% ...queries complete in 1.8 seconds       

total time: 6.32 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              0.82s
Cleaning                   1.12s
Filtering/Summarizing      4.38s
    Data quality           0.05s
    Clear day detect       0.46s
    Clipping detect        3.52s
    Capacity change detect 0.35s



In [9]:
dask_tool = SDTDask(pvdaq_data_plug, local_client)
pvdaq_keys = [(34, 2011), (35, 2015), (51,2012)]

dask_tool.execute(pvdaq_keys, solver="OSQP", solver_convex="OSQP", fix_shifts=True, verbose=True)

## AWS Fargate Client example
use LocalFile dataplug and fargate client

### Instantiate a localfile data plug

In [ ]:
from sdt_dask.dataplugs.csv_plug import LocalFiles

In [ ]:
LocalFiles?

In [7]:
path = "../dataplugs/example_data/"
local_file_data_plug = LocalFiles(path_to_files=path)
local_file_keys = [(os.path.basename(fname)[:-4],) for fname in glob.glob(path + "*")]
local_file_keys = local_file_keys[1:]
local_file_keys

[('TAAI01129193',), ('TABC01142170',), ('TABD01110568',)]

In [8]:
local_file_data_plug.get_data(local_file_keys[1])

Loading file TABC01142170...


,ac_power_01,ac_power_02,ac_power_03
ts,,,
2013-12-31 16:00:00,0.0767,0.0928,0.0928
2013-12-31 16:05:00,0.0695,0.0862,0.0819
2013-12-31 16:10:00,0.0626,0.0726,0.0653
2013-12-31 16:15:00,0.0546,0.0639,0.0613
2013-12-31 16:20:00,0.0418,0.0530,0.0516
...,...,...,...
2019-06-20 15:30:00,0.5872,0.5950,0.2445
2019-06-20 15:35:00,0.6454,0.6506,0.2748
2019-06-20 15:40:00,0.6220,0.6313,0.2645


### Set up a Fargate cluster client


In [ ]:
from sdt_dask.clients.aws.fargate import Fargate

In [9]:
# Use should define an environment variable for secret key
PA_NUMBER = os.getenv("project_pa_number")
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')
ENVIRONMENT = {
    'AWS_ACCESS_KEY_ID' : os.getenv('AWS_ACCESS_KEY_ID'),
    'AWS_SECRET_ACCESS_KEY' : os.getenv('AWS_SECRET_ACCESS_KEY')
}

In [10]:
# The Tag, VPC, image, workers, threads per worker and environment need to be user defined and passed to the client class
TAGS = {
    "project-pa-number": PA_NUMBER,
    "project": "pvinsight"
}
VPC = "vpc-ab2ff6d3" # for us-west-2
IMAGE = "nimishy/sdt-dask-windows:latest"

WORKERS = 3
THREADS_PER_WORKER = 1

In [11]:
fargate_client = Fargate().init_client(image=IMAGE, 
                               tags=TAGS, 
                               vpc=VPC, 
                               region_name=AWS_DEFAULT_REGION,
                               environment=ENVIRONMENT,
                               n_workers=WORKERS,
                               threads_per_worker=THREADS_PER_WORKER
                               )

[i] Initilializing Fargate Cluster ...


RuntimeError: Cluster failed to start: Timed out trying to connect to tcp://35.85.226.66:8786 after 30 s

In [ ]:
dask_tool = SDTDask(local_file_data_plug, fargate_client)

dask_tool.execute(local_file_keys, solver="OSQP", solver_convex="OSQP", fix_shifts=True, verbose=True)

## Azure Client example